### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [6]:
import zipfile
with zipfile.ZipFile("ml-latest-small.zip","r") as zip_ref:
    zip_ref.extractall("ml-latest-small")


In [7]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [8]:
joined = ratings.merge(movies, on='movieId', how='left')

In [ ]:
# Вариант 1

In [9]:
def class_rating(row):
    
    if row['rating'] <= 2:
        return 'low rating'
    elif 2 < row['rating'] <= 4:
         return 'average rating'
    elif 4.5 <= row['rating'] == 5:
        return 'high rating'

In [10]:
joined['class'] = joined.apply(class_rating, axis=1)

In [11]:
joined.head()

,userId,movieId,rating,timestamp,title,genres,class
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,average rating
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,average rating
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,average rating
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,low rating
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,average rating


In [ ]:
# Вариант 2

In [12]:
def class_rating(row):
    
    if row <= 2:
        return 'low rating'
    elif 2 < row <= 4:
         return 'average rating'
    elif 4.5 <= row == 5:
        return 'high rating'

In [13]:
joined['class'] = joined['rating'].apply(class_rating)
joined[ ['rating', 'title','class'] ].head()

,rating,title,class
0,2.5,Dangerous Minds (1995),average rating
1,3.0,Dumbo (1941),average rating
2,3.0,Sleepers (1996),average rating
3,2.0,Escape from New York (1981),low rating
4,4.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989),average rating


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

1
2
3
4
5
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [52]:
keywords = pd.read_csv('ml-latest-small/keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [86]:
def geo_region(keyword):
    geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
                'Северо-Запад': ['петербург', 'псков', 'мурманск'],
                'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}
    
    for reg,v in geo_data.items():
        for i in keyword.split(' '):
            if i in v:
                return reg
    return 'undefined'
    
    

In [87]:
keywords['region'] = keywords['keyword'].apply(geo_region)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [88]:
keywords[(keywords.region != 'undefined')].head(10)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
1259,москва,153050,Центр
1346,санкт петербург,144232,Северо-Запад
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
1579,авито тула,126168,Центр


In [89]:

keywords[keywords['keyword'] == 'сахалин']

,keyword,shows,region
21445,сахалин,14494,Дальний Восток


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [120]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
joined = ratings.merge(movies, on='movieId', how='left')

In [102]:
years = list(range(1950, 2011))

In [108]:
def production_year(movie):
    for year in years:
        if str(year) in str(movie):
            return year
    return 1900
            

In [151]:
joined['year'] = joined['year'].apply(production_year)


In [152]:
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [153]:
year_mean_rating = joined.groupby('year').mean().reset_index()[['year', 'rating']].sort_values('rating', ascending=False)

year_mean_rating

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
...,...,...
56,2005,3.448434
54,2003,3.444777
47,1996,3.426600
48,1997,3.415764
